<a href="https://colab.research.google.com/github/jucafernando/Projeto_pyspark/blob/main/Projeto_Pyspark_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [121]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Resolvendo problema de versão python x pyspark workers

In [122]:
from pyspark.sql import SparkSession  # importando a seção para iniciar o pyspark
from pyspark.sql.functions import *   # importando funções do pyspark
from pyspark.sql.types import *       # importando função no pyspark que determina qual o tipo de dado em uma coluna 

In [123]:
# Criando/Iniciar seção no PySpark
 
spark = (SparkSession.builder       # Vai criar a seção no spark
         .master('local')           # Como o pyspark trabalha com cluster, essa opção mostra em qual cluster será usado e o nome. 
         .appName('Pyspark_01')     # Nome da seção no spark
         .getOrCreate()             # Essa função puxará alguma seção que já foi criada. Se ainda não foi criada, será criada uma nova seção. 
         )


In [124]:

from google.colab import files
uploaded = files.upload()



Saving Emprestimo.csv to Emprestimo (3).csv
Saving Editora.csv to Editora (3).csv
Saving Autor.csv to Autor (3).csv
Saving Devolucao.csv to Devolucao (3).csv
Saving Departamento.csv to Departamento (3).csv
Saving Cargo.csv to Cargo (3).csv
Saving Estoque.csv to Estoque (3).csv
Saving Funcionario.csv to Funcionario (3).csv
Saving Obra.csv to Obra (3).csv
Saving Usuario.csv to Usuario (3).csv
Saving Reserva.csv to Reserva (3).csv


In [125]:
# Criando um dataframe e ler o arquivo 

df = spark.read.csv('Devolucao.csv', header=True, inferSchema=True)

In [126]:
df.show()

+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+
|ID_Devolucao|ID_Funcionario|ID_Emprestimo|ID_Estoque|ID_Obra|Data_Devolucao|     Hora_Devolucao|Multa_Atraso|
+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+
|           1|             1|            1|        21|     10|     17/8/2011|2022-08-16 18:00:00|          10|
|           2|             8|            2|        14|     14|     28/9/2011|2022-08-16 13:00:00|           0|
|           3|             8|            3|        11|     18|     13/7/2011|2022-08-16 14:55:00|           2|
|           4|             8|            4|        30|     17|     20/8/2011|2022-08-16 12:01:00|           3|
|           5|             1|            5|         1|     23|     11/6/2011|2022-08-16 10:00:00|           0|
|           6|             8|            6|         5|     24|     22/8/2011|2022-08-16 14:55:00|           4|
|

In [127]:
# Verificar o tipo de coluna

df.printSchema()

root
 |-- ID_Devolucao: integer (nullable = true)
 |-- ID_Funcionario: integer (nullable = true)
 |-- ID_Emprestimo: integer (nullable = true)
 |-- ID_Estoque: integer (nullable = true)
 |-- ID_Obra: integer (nullable = true)
 |-- Data_Devolucao: string (nullable = true)
 |-- Hora_Devolucao: timestamp (nullable = true)
 |-- Multa_Atraso: integer (nullable = true)



In [128]:
# Verificando dados nulos. Mandei transformar os dados para pandas, verificando se é nulo e fazer uma soma. 

df.toPandas().isna().sum

<bound method NDFrame._add_numeric_operations.<locals>.sum of     ID_Devolucao  ID_Funcionario  ID_Emprestimo  ID_Estoque  ID_Obra  \
0          False           False          False       False    False   
1          False           False          False       False    False   
2          False           False          False       False    False   
3          False           False          False       False    False   
4          False           False          False       False    False   
5          False           False          False       False    False   
6          False           False          False       False    False   
7          False           False          False       False    False   
8          False           False          False       False    False   
9          False           False          False       False    False   
10         False           False          False       False    False   
11         False           False          False       False    False   
12

In [129]:
# Forma de verificar se existe valores nulos sem precisar transformar para o Pandas:

for coluna in df.columns:
  print(coluna, df.filter(df[coluna].isNull()).count())

ID_Devolucao 0
ID_Funcionario 0
ID_Emprestimo 0
ID_Estoque 0
ID_Obra 0
Data_Devolucao 0
Hora_Devolucao 0
Multa_Atraso 0


In [130]:
# Renomeando colunas. O comando withColumnRenamed pode ser usado para esse procedimento:

df.withColumnRenamed('Data_Devolucao', 'data_devolucao').show()     # o .show é utilizado para mostrar o resultado, já podendo analisar se o procedimento deu certo ou não. 


+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+
|ID_Devolucao|ID_Funcionario|ID_Emprestimo|ID_Estoque|ID_Obra|data_devolucao|     Hora_Devolucao|Multa_Atraso|
+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+
|           1|             1|            1|        21|     10|     17/8/2011|2022-08-16 18:00:00|          10|
|           2|             8|            2|        14|     14|     28/9/2011|2022-08-16 13:00:00|           0|
|           3|             8|            3|        11|     18|     13/7/2011|2022-08-16 14:55:00|           2|
|           4|             8|            4|        30|     17|     20/8/2011|2022-08-16 12:01:00|           3|
|           5|             1|            5|         1|     23|     11/6/2011|2022-08-16 10:00:00|           0|
|           6|             8|            6|         5|     24|     22/8/2011|2022-08-16 14:55:00|           4|
|

In [131]:
# No spark, o dataframe é imutável, ou seja, depois de criado, não poderá ser alterado. Uma solução para isso seria criar uma nova versão do dataframe:

df = df.withColumnRenamed('Data_Devolucao', 'data_devolucao')

df.show()

+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+
|ID_Devolucao|ID_Funcionario|ID_Emprestimo|ID_Estoque|ID_Obra|data_devolucao|     Hora_Devolucao|Multa_Atraso|
+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+
|           1|             1|            1|        21|     10|     17/8/2011|2022-08-16 18:00:00|          10|
|           2|             8|            2|        14|     14|     28/9/2011|2022-08-16 13:00:00|           0|
|           3|             8|            3|        11|     18|     13/7/2011|2022-08-16 14:55:00|           2|
|           4|             8|            4|        30|     17|     20/8/2011|2022-08-16 12:01:00|           3|
|           5|             1|            5|         1|     23|     11/6/2011|2022-08-16 10:00:00|           0|
|           6|             8|            6|         5|     24|     22/8/2011|2022-08-16 14:55:00|           4|
|

In [132]:
# Selecionando colunas:

df.select('Id_Devolucao', 'Id_Funcionario')

DataFrame[Id_Devolucao: int, Id_Funcionario: int]

In [133]:
# Outras formas de selecionar colunas:

df.select(col('Hora_Devolucao').alias('Hora da Devolução'), col('Multa_Atraso').alias('Multa por Atraso')).show()     # incluindo alias(apelidos)

+-------------------+----------------+
|  Hora da Devolução|Multa por Atraso|
+-------------------+----------------+
|2022-08-16 18:00:00|              10|
|2022-08-16 13:00:00|               0|
|2022-08-16 14:55:00|               2|
|2022-08-16 12:01:00|               3|
|2022-08-16 10:00:00|               0|
|2022-08-16 14:55:00|               4|
|2022-08-16 13:00:00|               0|
|2022-08-16 14:55:00|               0|
|2022-08-16 10:00:00|               1|
|2022-08-16 18:00:00|               2|
|2022-08-16 14:29:00|               0|
|2022-08-16 17:54:00|               1|
|2022-08-16 08:36:00|               0|
|2022-08-16 16:30:00|               0|
|2022-08-16 11:11:00|               1|
|2022-08-16 10:00:00|               0|
|2022-08-16 18:00:00|               6|
|2022-08-16 15:00:00|               1|
|2022-08-16 16:00:00|               0|
|2022-08-16 18:00:00|               3|
+-------------------+----------------+
only showing top 20 rows



In [134]:
# Uma outra forma de fazer um select, é colocando o nome do dataframe. Dessa forma:

df.select(df['Id_Devolucao'].alias('Id de Devolução'), df['Id_Funcionario'].alias('Id do Funcionário'))

DataFrame[Id de Devolução: int, Id do Funcionário: int]

In [135]:
# Organizando um select:

df.select(col('Multa_Atraso'), col('Data_Devolucao')).show()

# Assim como no SQL, é possível mandar selecionar as colunas na ordem que desejar. 

+------------+--------------+
|Multa_Atraso|Data_Devolucao|
+------------+--------------+
|          10|     17/8/2011|
|           0|     28/9/2011|
|           2|     13/7/2011|
|           3|     20/8/2011|
|           0|     11/6/2011|
|           4|     22/8/2011|
|           0|     27/3/2012|
|           0|     21/8/2011|
|           1|      3/4/2012|
|           2|     21/8/2011|
|           0|     31/3/2012|
|           1|     23/8/2011|
|           0|      2/4/2012|
|           0|      3/4/2012|
|           1|     30/6/2013|
|           0|     22/8/2011|
|           6|     22/8/2011|
|           1|     15/7/2013|
|           0|      9/7/2013|
|           3|     27/6/2013|
+------------+--------------+
only showing top 20 rows



In [136]:
#Realizando filtros. Para isso, usa-se o parametro df.filter:
         
df.orderBy(['Hora_devolucao'], ascending=True).show()

+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+
|ID_Devolucao|ID_Funcionario|ID_Emprestimo|ID_Estoque|ID_Obra|data_devolucao|     Hora_Devolucao|Multa_Atraso|
+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+
|          13|             1|           13|        29|     16|      2/4/2012|2022-08-16 08:36:00|           0|
|           9|             8|            9|        10|     30|      3/4/2012|2022-08-16 10:00:00|           1|
|           5|             1|            5|         1|     23|     11/6/2011|2022-08-16 10:00:00|           0|
|          16|             1|           16|        13|     13|     22/8/2011|2022-08-16 10:00:00|           0|
|          28|            11|           28|        12|     29|      2/2/2013|2022-08-16 10:00:00|           1|
|          15|             8|           15|        23|     26|     30/6/2013|2022-08-16 11:11:00|           1|
|

In [137]:
# Filtrando DataFrames com a condição AND (&)

df.filter((col('Id_Funcionario') == 1) & (col('Hora_Devolucao') >= '2022-08-15')).show()

+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+
|ID_Devolucao|ID_Funcionario|ID_Emprestimo|ID_Estoque|ID_Obra|data_devolucao|     Hora_Devolucao|Multa_Atraso|
+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+
|           1|             1|            1|        21|     10|     17/8/2011|2022-08-16 18:00:00|          10|
|           5|             1|            5|         1|     23|     11/6/2011|2022-08-16 10:00:00|           0|
|           7|             1|            7|         7|     21|     27/3/2012|2022-08-16 13:00:00|           0|
|          10|             1|           10|        27|      9|     21/8/2011|2022-08-16 18:00:00|           2|
|          12|             1|           12|         3|     25|     23/8/2011|2022-08-16 17:54:00|           1|
|          13|             1|           13|        29|     16|      2/4/2012|2022-08-16 08:36:00|           0|
|

In [138]:
df.show()

+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+
|ID_Devolucao|ID_Funcionario|ID_Emprestimo|ID_Estoque|ID_Obra|data_devolucao|     Hora_Devolucao|Multa_Atraso|
+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+
|           1|             1|            1|        21|     10|     17/8/2011|2022-08-16 18:00:00|          10|
|           2|             8|            2|        14|     14|     28/9/2011|2022-08-16 13:00:00|           0|
|           3|             8|            3|        11|     18|     13/7/2011|2022-08-16 14:55:00|           2|
|           4|             8|            4|        30|     17|     20/8/2011|2022-08-16 12:01:00|           3|
|           5|             1|            5|         1|     23|     11/6/2011|2022-08-16 10:00:00|           0|
|           6|             8|            6|         5|     24|     22/8/2011|2022-08-16 14:55:00|           4|
|

In [139]:
# Filtrando DataFrames com a condição OR (/)

df.filter((col('Id_Funcionario') == 1) | (col('data_devolucao') == '30/06/2013') | (col('Hora_Devolucao') >= '2022-08-15')).show()

+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+
|ID_Devolucao|ID_Funcionario|ID_Emprestimo|ID_Estoque|ID_Obra|data_devolucao|     Hora_Devolucao|Multa_Atraso|
+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+
|           1|             1|            1|        21|     10|     17/8/2011|2022-08-16 18:00:00|          10|
|           2|             8|            2|        14|     14|     28/9/2011|2022-08-16 13:00:00|           0|
|           3|             8|            3|        11|     18|     13/7/2011|2022-08-16 14:55:00|           2|
|           4|             8|            4|        30|     17|     20/8/2011|2022-08-16 12:01:00|           3|
|           5|             1|            5|         1|     23|     11/6/2011|2022-08-16 10:00:00|           0|
|           6|             8|            6|         5|     24|     22/8/2011|2022-08-16 14:55:00|           4|
|

In [140]:
# Filtrando um Dataframe combinando &(AND) e |(OR)

df.filter((col('Id_Funcionario') == 1) & (col('data_devolucao') == '22/8/2011') | (col('Id_Funcionario') == 11) & (col('data_devolucao') == '15/7/2013')).show() 

+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+
|ID_Devolucao|ID_Funcionario|ID_Emprestimo|ID_Estoque|ID_Obra|data_devolucao|     Hora_Devolucao|Multa_Atraso|
+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+
|          16|             1|           16|        13|     13|     22/8/2011|2022-08-16 10:00:00|           0|
|          17|             1|           17|        15|     19|     22/8/2011|2022-08-16 18:00:00|           6|
|          18|            11|           18|        16|      4|     15/7/2013|2022-08-16 15:00:00|           1|
+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+



In [141]:
# Criar novas colunas usando lit

df.withColumn('Porcentagem_Multa', lit(15.5)).show()

+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+-----------------+
|ID_Devolucao|ID_Funcionario|ID_Emprestimo|ID_Estoque|ID_Obra|data_devolucao|     Hora_Devolucao|Multa_Atraso|Porcentagem_Multa|
+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+-----------------+
|           1|             1|            1|        21|     10|     17/8/2011|2022-08-16 18:00:00|          10|             15.5|
|           2|             8|            2|        14|     14|     28/9/2011|2022-08-16 13:00:00|           0|             15.5|
|           3|             8|            3|        11|     18|     13/7/2011|2022-08-16 14:55:00|           2|             15.5|
|           4|             8|            4|        30|     17|     20/8/2011|2022-08-16 12:01:00|           3|             15.5|
|           5|             1|            5|         1|     23|     11/6/2011|2022-08-16 10:00:00|

In [142]:
df.withColumn('Valor_Multa', lit(col('Multa_Atraso') + 0.10)).show()

+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+-----------+
|ID_Devolucao|ID_Funcionario|ID_Emprestimo|ID_Estoque|ID_Obra|data_devolucao|     Hora_Devolucao|Multa_Atraso|Valor_Multa|
+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+-----------+
|           1|             1|            1|        21|     10|     17/8/2011|2022-08-16 18:00:00|          10|       10.1|
|           2|             8|            2|        14|     14|     28/9/2011|2022-08-16 13:00:00|           0|        0.1|
|           3|             8|            3|        11|     18|     13/7/2011|2022-08-16 14:55:00|           2|        2.1|
|           4|             8|            4|        30|     17|     20/8/2011|2022-08-16 12:01:00|           3|        3.1|
|           5|             1|            5|         1|     23|     11/6/2011|2022-08-16 10:00:00|           0|        0.1|
|           6|  

In [143]:
df.show()

+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+
|ID_Devolucao|ID_Funcionario|ID_Emprestimo|ID_Estoque|ID_Obra|data_devolucao|     Hora_Devolucao|Multa_Atraso|
+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+
|           1|             1|            1|        21|     10|     17/8/2011|2022-08-16 18:00:00|          10|
|           2|             8|            2|        14|     14|     28/9/2011|2022-08-16 13:00:00|           0|
|           3|             8|            3|        11|     18|     13/7/2011|2022-08-16 14:55:00|           2|
|           4|             8|            4|        30|     17|     20/8/2011|2022-08-16 12:01:00|           3|
|           5|             1|            5|         1|     23|     11/6/2011|2022-08-16 10:00:00|           0|
|           6|             8|            6|         5|     24|     22/8/2011|2022-08-16 14:55:00|           4|
|

In [144]:
# Criando uma coluna condicional (usando a função substring). Pode ser usado para abreviar dados como por exemplo: 

df.withColumn('Ano_Hora_Devolucao', substring('Hora_Devolucao', 1, 10)).show()

+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+------------------+
|ID_Devolucao|ID_Funcionario|ID_Emprestimo|ID_Estoque|ID_Obra|data_devolucao|     Hora_Devolucao|Multa_Atraso|Ano_Hora_Devolucao|
+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+------------------+
|           1|             1|            1|        21|     10|     17/8/2011|2022-08-16 18:00:00|          10|        2022-08-16|
|           2|             8|            2|        14|     14|     28/9/2011|2022-08-16 13:00:00|           0|        2022-08-16|
|           3|             8|            3|        11|     18|     13/7/2011|2022-08-16 14:55:00|           2|        2022-08-16|
|           4|             8|            4|        30|     17|     20/8/2011|2022-08-16 12:01:00|           3|        2022-08-16|
|           5|             1|            5|         1|     23|     11/6/2011|2022-08-16 10

In [145]:
# O dataframe por defalt, não sobrepoe uma coluna sobre a outra. Então a solução para manter as alterações feitas acima é criando um dataframe sobre o dataframe anterior:

In [146]:
df = df.withColumn('Ano_Hora_Devolucao', substring('Hora_Devolucao', 1, 10))

In [147]:
df.show()

+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+------------------+
|ID_Devolucao|ID_Funcionario|ID_Emprestimo|ID_Estoque|ID_Obra|data_devolucao|     Hora_Devolucao|Multa_Atraso|Ano_Hora_Devolucao|
+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+------------------+
|           1|             1|            1|        21|     10|     17/8/2011|2022-08-16 18:00:00|          10|        2022-08-16|
|           2|             8|            2|        14|     14|     28/9/2011|2022-08-16 13:00:00|           0|        2022-08-16|
|           3|             8|            3|        11|     18|     13/7/2011|2022-08-16 14:55:00|           2|        2022-08-16|
|           4|             8|            4|        30|     17|     20/8/2011|2022-08-16 12:01:00|           3|        2022-08-16|
|           5|             1|            5|         1|     23|     11/6/2011|2022-08-16 10

In [148]:
# Criar colunas condicionais (concat/concat_ws). Primeiro utilizando a forma contat:

df.withColumn('Concat', concat('data_Devolucao', col('Multa_Atraso'))).show()

+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+------------------+-----------+
|ID_Devolucao|ID_Funcionario|ID_Emprestimo|ID_Estoque|ID_Obra|data_devolucao|     Hora_Devolucao|Multa_Atraso|Ano_Hora_Devolucao|     Concat|
+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+------------------+-----------+
|           1|             1|            1|        21|     10|     17/8/2011|2022-08-16 18:00:00|          10|        2022-08-16|17/8/201110|
|           2|             8|            2|        14|     14|     28/9/2011|2022-08-16 13:00:00|           0|        2022-08-16| 28/9/20110|
|           3|             8|            3|        11|     18|     13/7/2011|2022-08-16 14:55:00|           2|        2022-08-16| 13/7/20112|
|           4|             8|            4|        30|     17|     20/8/2011|2022-08-16 12:01:00|           3|        2022-08-16| 20/8/20113|
|     

In [149]:
# Outra forma é pelo concat_ws. Essa forma é mais adequada pois conseguimos deixar os dados mais organizados:

df.withColumn('Separador', concat_ws(' - ', 'data_Devolucao','Multa_Atraso')).show()

+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+------------------+--------------+
|ID_Devolucao|ID_Funcionario|ID_Emprestimo|ID_Estoque|ID_Obra|data_devolucao|     Hora_Devolucao|Multa_Atraso|Ano_Hora_Devolucao|     Separador|
+------------+--------------+-------------+----------+-------+--------------+-------------------+------------+------------------+--------------+
|           1|             1|            1|        21|     10|     17/8/2011|2022-08-16 18:00:00|          10|        2022-08-16|17/8/2011 - 10|
|           2|             8|            2|        14|     14|     28/9/2011|2022-08-16 13:00:00|           0|        2022-08-16| 28/9/2011 - 0|
|           3|             8|            3|        11|     18|     13/7/2011|2022-08-16 14:55:00|           2|        2022-08-16| 13/7/2011 - 2|
|           4|             8|            4|        30|     17|     20/8/2011|2022-08-16 12:01:00|           3|        2022-08-16| 

In [150]:
# Alterando o tipo de coluna 

df.withColumn('Hora_Devolucao', col('Hora_Devolucao').cast(IntegerType())).printSchema()

root
 |-- ID_Devolucao: integer (nullable = true)
 |-- ID_Funcionario: integer (nullable = true)
 |-- ID_Emprestimo: integer (nullable = true)
 |-- ID_Estoque: integer (nullable = true)
 |-- ID_Obra: integer (nullable = true)
 |-- data_devolucao: string (nullable = true)
 |-- Hora_Devolucao: integer (nullable = true)
 |-- Multa_Atraso: integer (nullable = true)
 |-- Ano_Hora_Devolucao: string (nullable = true)



In [151]:
df.printSchema()

# Feito o procedimento, verificamos que a alteração de coluna não foi feita de maneira definitiva. 

root
 |-- ID_Devolucao: integer (nullable = true)
 |-- ID_Funcionario: integer (nullable = true)
 |-- ID_Emprestimo: integer (nullable = true)
 |-- ID_Estoque: integer (nullable = true)
 |-- ID_Obra: integer (nullable = true)
 |-- data_devolucao: string (nullable = true)
 |-- Hora_Devolucao: timestamp (nullable = true)
 |-- Multa_Atraso: integer (nullable = true)
 |-- Ano_Hora_Devolucao: string (nullable = true)



In [152]:
# Para resolver essa questão, é preciso criar um dataframe sobre o dataframe de onde precisamos alterar a coluna: 


df = df.withColumn('Hora_Devolucao', col('Hora_Devolucao').cast(IntegerType()))

In [153]:
# Verificando de as alterações foram realizadas na tabela:

df.printSchema()

# Alteração bem sucedida. 

root
 |-- ID_Devolucao: integer (nullable = true)
 |-- ID_Funcionario: integer (nullable = true)
 |-- ID_Emprestimo: integer (nullable = true)
 |-- ID_Estoque: integer (nullable = true)
 |-- ID_Obra: integer (nullable = true)
 |-- data_devolucao: string (nullable = true)
 |-- Hora_Devolucao: integer (nullable = true)
 |-- Multa_Atraso: integer (nullable = true)
 |-- Ano_Hora_Devolucao: string (nullable = true)



In [154]:
df.withColumn('data_Devolucao', col('data_Devolucao').cast(DateType())).printSchema()

root
 |-- ID_Devolucao: integer (nullable = true)
 |-- ID_Funcionario: integer (nullable = true)
 |-- ID_Emprestimo: integer (nullable = true)
 |-- ID_Estoque: integer (nullable = true)
 |-- ID_Obra: integer (nullable = true)
 |-- data_Devolucao: date (nullable = true)
 |-- Hora_Devolucao: integer (nullable = true)
 |-- Multa_Atraso: integer (nullable = true)
 |-- Ano_Hora_Devolucao: string (nullable = true)



In [155]:
df.show()

+------------+--------------+-------------+----------+-------+--------------+--------------+------------+------------------+
|ID_Devolucao|ID_Funcionario|ID_Emprestimo|ID_Estoque|ID_Obra|data_devolucao|Hora_Devolucao|Multa_Atraso|Ano_Hora_Devolucao|
+------------+--------------+-------------+----------+-------+--------------+--------------+------------+------------------+
|           1|             1|            1|        21|     10|     17/8/2011|    1660672800|          10|        2022-08-16|
|           2|             8|            2|        14|     14|     28/9/2011|    1660654800|           0|        2022-08-16|
|           3|             8|            3|        11|     18|     13/7/2011|    1660661700|           2|        2022-08-16|
|           4|             8|            4|        30|     17|     20/8/2011|    1660651260|           3|        2022-08-16|
|           5|             1|            5|         1|     23|     11/6/2011|    1660644000|           0|        2022-08-16|
